In [ ]:
using Revise

In [ ]:
using MeshCatMechanisms
using MeshCat
using RigidBodyDynamics
using Gurobi

In [ ]:
import LCPSim
import LearningMPC
import BoxValkyries
import Nets
reload("LearningMPC")
reload("BoxValkyries")

In [ ]:
robot = BoxValkyries.BoxValkyrie()

In [ ]:
mvis = MechanismVisualizer(robot)
IJuliaCell(mvis)

In [ ]:

xstar = BoxValkyries.nominal_state(robot)

mpc_params = LearningMPC.MPCParams(
    Δt=0.05,
    horizon=10,
    mip_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0, TimeLimit=120, MIPGap=1e-1, FeasibilityTol=1e-3),
    lcp_solver=GurobiSolver(Gurobi.Env(), OutputFlag=0))

Q, R = BoxValkyries.default_costs(robot)
feet = findbody.(robot.mechanism, ["lf", "rf"])
lqrsol = LearningMPC.LQRSolution(xstar, Q, R, mpc_params.Δt, Point3D.(default_frame.(feet), 0., 0., 0.))
lqrsol.S .= 1 ./ mpc_params.Δt .* Q

mpc_controller = LearningMPC.MPCController(robot.mechanism, 
    robot.environment, mpc_params, lqrsol, 
    [lqrsol]);

sample_sink = LearningMPC.MPCSampleSink{Float64}(true)
playback_sink = LearningMPC.PlaybackSink(mvis, mpc_params.Δt)

mpc_controller.callback = LearningMPC.call_each(
    sample_sink,
    playback_sink,
)


In [ ]:
playback_sink.last_trajectory;

In [ ]:
LearningMPC.playback(mvis, playback_sink.last_trajectory, mpc_params.Δt)

In [ ]:
x0 = BoxValkyries.nominal_state(robot)
set_configuration!(x0, configuration(x0) .+ 0.1 .* randn(length(configuration(x0))))
mpc_controller(x0)